In [34]:
# Script by Lars Kjær

# Source: https://careerfoundry.com/en/blog/data-analytics/how-to-find-outliers/

##################### Import libraries ################################

import pandas as pd
import os
import re

In [35]:
os.getcwd()

'C:\\Users\\lakj\\Documents\\GitHub\\isds2023\\Lars'

In [36]:
# Import data
df = pd.read_csv(r'.\clean_data\processed_data_sales_1992_2022_161.csv')

In [37]:
df.describe()

,municipality,zip_code,purchase_amount,square meters,square meters_price,rooms,year_of_construction
count,9207.0,9207.0,9.207000e+03,9207.000000,9207.000000,9207.000000,9207.000000
mean,161.0,2600.0,2.968808e+06,104.614750,35838.957967,3.681112,1960.700554
std,0.0,0.0,5.473231e+06,43.924268,83084.660284,1.584664,50.291863
min,161.0,2600.0,1.750000e+04,9.000000,222.000000,1.000000,0.000000
25%,161.0,2600.0,9.000000e+05,70.000000,9787.500000,3.000000,1940.000000
50%,161.0,2600.0,1.625000e+06,98.000000,16489.000000,4.000000,1959.000000
75%,161.0,2600.0,2.760000e+06,134.000000,25405.000000,5.000000,1975.000000
max,161.0,2600.0,6.347990e+07,388.000000,783784.000000,14.000000,2023.000000


These are a few of the most popular visualization methods for finding outliers in data:

- Histogram
- Box plot
- Scatter plot


Using a histogram, we can see how the data is distributed. Having data that follows a normal distribution is necessary for some of the statistical techniques used to detect outliers. If the data doesn’t follow a normal distribution, the z-score calculation shouldn’t be used to find the outliers. 

In [38]:
import plotly.express as px
#create a histogram

fig = px.histogram(df, x='square meters_price')

fig.show()

Notice the data does not follow a normal distribution. Since the data is skewed, instead of using a z-score we can use **interquartile range (IQR)** to determine the outliers. We will explore using IQR after reviewing the other visualization techniques.

# Find outliers in data using a box plot


In [39]:
#create a box plot

fig = px.box(df, y='square meters_price')

fig.show()

# Find multivariate outliers using a scatter plot

In [40]:
fig = px.scatter(x=df['rooms'], y=df['square meters_price'])

fig.show()

# Finding outliers using statistical methods

In [41]:
# we will calculate the outlier data points using the statistical method called interquartile range (IQR) instead of using Z-score. Using the IQR, the outlier data points are the ones falling below Q1–1.5 IQR or above Q3 + 1.5 IQR. The Q1 is the 25th percentile and Q3 is the 75th percentile of the dataset, and IQR represents the interquartile range calculated by Q3 minus Q1 (Q3–Q1). 

#Using the convenient pandas .quantile() function, we can create a simple Python function that takes in our column from the dataframe and outputs the outliers:

#create a function to find outliers using IQR

def find_outliers_IQR(df):

   q1=df.quantile(0.25)

   q3=df.quantile(0.75)

   IQR=q3-q1

   outliers = df[((df<(q1-1.5*IQR)) | (df>(q3+1.5*IQR)))]

   return outliers


outliers = find_outliers_IQR(df['square meters_price'])

print("number of outliers: "+ str(len(outliers)))

print("max outlier value: "+ str(outliers.max()))

print("min outlier value: "+ str(outliers.min()))

number of outliers: 717
max outlier value: 783784
min outlier value: 48913


# Working with outliers using statistical methods

After identifying the outliers, we need to decide what to do with them. Unfortunately, there is no straightforward “best” solution for dealing with outliers because it depends on the severity of outliers and the goals of the analysis. For example, since we think the value 208 in the passenger_count was caused by human error, we should treat that outlier differently than the outliers for fare_amount. Here are three techniques we can use to handle outliers:

- Drop the outliers
- Cap the outliers
- Replace outliers using imputation as if they were missing values

I’ll go over those in detail now.

## Drop the outliers
Using this method, we essentially drop all the outliers from the data, excluding them from the analysis and modeling. Although this technique is quick and easy, it isn’t always the right solution and can reduce the amount of data if there are a lot of outliers present. 

Copy and paste the find_outliers_IQR function **( IQR = interquartile range )** so we can modify it to return a dataframe with the outliers removed. Rename it drop_outliers_IQR . Inside the function we create a dataframe named not_outliers that replaces the outlier values with a NULL. Then we can use .dropna(), to drop the rows with NULL values. 

In [42]:
def drop_outliers_IQR(df):

   q1=df.quantile(0.25)

   q3=df.quantile(0.75)

   IQR=q3-q1

   not_outliers = df[~((df<(q1-1.5*IQR)) | (df>(q3+1.5*IQR)))]

   outliers_dropped = outliers.dropna().reset_index()

   return outliers_dropped, 'rows dropped: ', len(outliers_dropped), 'rows before: ', len(df)

new_df = df.copy()
drop_outliers_IQR(new_df)

dropped_outl = drop_outliers_IQR(new_df)[0]

C:\Users\lakj\AppData\Local\Temp\ipykernel_11320\2064691818.py:3: FutureWarning:

The default value of numeric_only in DataFrame.quantile is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.

C:\Users\lakj\AppData\Local\Temp\ipykernel_11320\2064691818.py:5: FutureWarning:

The default value of numeric_only in DataFrame.quantile is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.

C:\Users\lakj\AppData\Local\Temp\ipykernel_11320\2064691818.py:9: FutureWarning:

Automatic reindexing on DataFrame vs Series comparisons is deprecated and will raise ValueError in a future version. Do `left, right = left.align(right, axis=1, copy=False)` before e.g. `left == right`

C:\Users\lakj\AppData\Local\Temp\ipykernel_11320\2064691818.py:3: FutureWarning:

The default value of numeric_only in DataFrame.quantile

In [43]:
dropped_outl

,index,square meters_price
0,101,52857
1,107,52151
2,108,83621
3,168,134247
4,169,136111
...,...,...
712,8783,184272
713,8784,184272
714,8785,180795
715,8786,180795


In [44]:
# Drop multiple rows using index labels
new_df = df.copy()
df1 = new_df.drop(list(dropped_outl['index']))

In [45]:
df1

,municipality,housing_type,address,zip_code,town,purchase_amount,date_of_sale,type_of_sale,square meters,square meters_price,rooms,year_of_construction,The percentage difference between the latest bid price and the sale price
0,161,Ejerlejlighed,"Dalvangsvej 13, 1. 3",2600,Glostrup,1000000,2022-12-28,Fam. Salg,88,11364,3,1967,NaN
1,161,Ejerlejlighed,"Kochsvej 5, st",2600,Glostrup,600000,2022-12-28,Fam. Salg,99,6061,3,1906,NaN
2,161,Ejerlejlighed,"Kochsvej 5, 1",2600,Glostrup,600000,2022-12-28,Fam. Salg,73,8219,3,1906,NaN
3,161,Rækkehus,Anemonevej 27,2600,Glostrup,3800000,2022-12-22,Alm. Salg,136,27941,4,1974,-5%
4,161,Villa,Statenevej 17,2600,Glostrup,2900000,2022-12-22,Alm. Salg,121,23967,4,1979,-9%
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9202,161,Villa,Sofielundsvej 5,2600,Glostrup,585550,1992-01-05,Alm. Salg,129,4539,6,1933,NaN
9203,161,Villa,Østerager 25,2600,Glostrup,1280000,1992-01-04,Alm. Salg,160,8000,5,1987,NaN
9204,161,Villa,Ejby Mosevej 43,2600,Glostrup,880000,1992-01-04,Alm. Salg,132,6667,4,1972,NaN
9205,161,Villa,Solsortevej 27,2600,Glostrup,624754,1992-01-03,Alm. Salg,83,7527,3,1954,NaN


# create a box plot after dropping the outliers  

In [46]:
#create a box plot

fig = px.box(df1, y='square meters_price')

fig.show()

## Cap the outliers
In this technique, we essentially set a limit for the min and max outlier values. Anything above or below the cap gets set to the capped min or max respectively. For example, if we set the cap max for fare_amount at 20, any outlier above 20 will be set to 20. This technique is used when you can assume that all outliers express the same behaviors or patterns, meaning the model wouldn’t learn anything new by allowing the outliers to remain. 

To cap the outliers, calculate a upper limit and lower limit. For the upper limit, we will use the mean plus three standard deviations. For the lower limit, we will calculate it as the mean minus 3 standard deviations. Keep in mind, the calculation you use can depend on the data’s distribution. 

In [47]:
new_df = df.copy()
upper_limit = new_df['square meters_price'].mean() + 3*new_df['square meters_price'].std()

print(upper_limit)

lower_limit = new_df['square meters_price'].mean() - 3*new_df['square meters_price'].std()

print(lower_limit)

285092.93881894666
-213415.02288541783


Based on our calculated limits, any outliers above 285092.93881894666 will be set to 285092.93881894666. Likewise, any outlier below -213415.02288541783 will be set to -213415.02288541783.

After calculating the upper and lower limit, we use the numpy .where() function to apply the limits to fare_amount. 

In [48]:
import numpy as np
new_df['square meters_price'] = np.where(new_df['square meters_price'] > upper_limit,

   upper_limit,

   np.where(

       new_df['square meters_price'] < lower_limit,

       lower_limit,

       new_df['square meters_price']

   )

)

In [52]:
new_df.describe()

,municipality,zip_code,purchase_amount,square meters,square meters_price,rooms,year_of_construction
count,9207.0,9207.0,9.207000e+03,9207.000000,9207.000000,9207.000000,9207.000000
mean,161.0,2600.0,2.968808e+06,104.614750,30615.404064,3.681112,1960.700554
std,0.0,0.0,5.473231e+06,43.924268,54815.979209,1.584664,50.291863
min,161.0,2600.0,1.750000e+04,9.000000,222.000000,1.000000,0.000000
25%,161.0,2600.0,9.000000e+05,70.000000,9787.500000,3.000000,1940.000000
50%,161.0,2600.0,1.625000e+06,98.000000,16489.000000,4.000000,1959.000000
75%,161.0,2600.0,2.760000e+06,134.000000,25405.000000,5.000000,1975.000000
max,161.0,2600.0,6.347990e+07,388.000000,285092.938819,14.000000,2023.000000


In [53]:
df.describe()

,municipality,zip_code,purchase_amount,square meters,square meters_price,rooms,year_of_construction
count,9207.0,9207.0,9.207000e+03,9207.000000,9207.000000,9207.000000,9207.000000
mean,161.0,2600.0,2.968808e+06,104.614750,35838.957967,3.681112,1960.700554
std,0.0,0.0,5.473231e+06,43.924268,83084.660284,1.584664,50.291863
min,161.0,2600.0,1.750000e+04,9.000000,222.000000,1.000000,0.000000
25%,161.0,2600.0,9.000000e+05,70.000000,9787.500000,3.000000,1940.000000
50%,161.0,2600.0,1.625000e+06,98.000000,16489.000000,4.000000,1959.000000
75%,161.0,2600.0,2.760000e+06,134.000000,25405.000000,5.000000,1975.000000
max,161.0,2600.0,6.347990e+07,388.000000,783784.000000,14.000000,2023.000000


# Replace outliers using imputation as if they were missing values

The third technique for handling outliers is similar to capping the values. Instead of using a capping calculation, use whatever imputation technique is being used on the missing values. For example, if the **square meters_price** column had missing values, we might find it appropriate to fill in the missing values using the mean. Since that is how we treat the missing values, we would do the same thing for the outliers. 

Use a function to find the outliers using IQR and replace them with the mean value. Name it impute_outliers_IQR. In the function, we can get an upper limit and a lower limit using the .max() and .min() functions respectively. Then we can use numpy .where() to replace the values like we did in the previous example.

In [54]:
def impute_outliers_IQR(df):

   q1=df.quantile(0.25)

   q3=df.quantile(0.75)

   IQR=q3-q1

   upper = df[~(df>(q3+1.5*IQR))].max()

   lower = df[~(df<(q1-1.5*IQR))].min()

   df = np.where(df > upper,

       df.mean(),

       np.where(

           df < lower,

           df.mean(),

           df

           )

       )

   return df

We can pass fare_amount through the impute_outliers_IQR function to transform the outliers into the mean value. We can use .describe() to verify the function works.

In [55]:
new_df = df.copy()
new_df['square meters_price'] = impute_outliers_IQR(new_df['square meters_price'])

new_df.describe()['square meters_price']

count     9207.000000
mean     18078.597682
std      10023.256399
min        222.000000
25%       9787.500000
50%      16489.000000
75%      25405.000000
max      48333.000000
Name: square meters_price, dtype: float64

## Wrapping up 
As we’ve seen, finding and handling outliers can be a complicated process. Luckily Python has libraries that make it easy to visualize and munge the data. We started by using box plots and scatter plots to analyze univariate and multivariate outliers. Then we used the interquartile range (IQR) calculation to find the data points in our skewed data. Lastly we tried three different feature engineering techniques to handle the outliers in the dataset.

Remember, sometimes leaving out the outliers in the data is acceptable and other times they can negatively impact analysis and modeling so they should be dealt with by feature engineering. It all depends on the goals of the analysis and the severity of the outliers.

You may also be interested in this online workshop we held on outliers with data scientist Dana Daskalova: https://youtu.be/dKjHd7i-jB4

In [56]:
#create a box plot

fig = px.box(new_df, y='square meters_price')

fig.show()

In [57]:
fig = px.scatter(x=new_df['rooms'], y=new_df['square meters_price'])

fig.show()

In [58]:
#create a histogram

fig = px.histogram(new_df, x='square meters_price')

fig.show()